### 哈希表
1. 哈希表内储存元素的桶均匀增长，即每个桶内有相似链长度的节点（相同hash值存放相同桶内）
2. 当一个桶内的链长度增长到一定长度，哈希表开始扩容，即增加桶的个数。目的使哈希表查询保持O(1)增删改查
3. x2扩容，每次扩容代价为O(N)，插入N个数，扩容整体代价趋近O(2N),平均下来是O(1)。

### 布隆过滤器(类黑名单系统，一定存在失误率(假阳性)，通过设计调节降低)
1. 利用哈希函数的性质
2. 每一条数据提取特征(多个哈希函数映射）
3. 加入描黑库
4. (HDFS、redis中有应用) HDFS每个存储块存一个自己的布隆过滤器

In [1]:
'''
1. 将所有非法url通过K个哈希函数，映射到M长度的位图上，将每个哈希输出描黑。 ==> 创建
2. 需要检测的一个新的url，通过相同K个哈希函数，将哈希输出的状态取出，这K个结结果中有一个未描黑即可判断不是非法url。 ==> 检索

巧妙点是 确定 K、M的大小：M为bit的个数
1. M的大小考虑相关：样本量N、失误率P。 M越大失误率越低，通过失误率P找到满足条件的M即可，无需无止境的扩大M，因为失误率无法=0，只能趋近。
M =  -(N * lnP)
    -----------
       (ln2)^2
2. K个函数的个数相关：样本量、M空间。初始K=1，P很大，随着K增大P会先变小，然后增大。增大是因为K增加使M的利用率增加，使失误率增加。
所以是个凹函数，取拐点即可获得最佳K值。
K = ln2 * M/N = 0.7 * M/N

3. 真实P的计算方式：
P = (1-e^(-N*K/M))^K

4.通过2个哈希函数造K个：任意整数i * f1 + f2即可

'''

'\n1. 将所有非法url通过K个哈希函数，映射到M长度的位图上，将每个哈希输出描黑。 ==> 创建\n2. 需要检测的一个新的url，通过相同K个哈希函数，将哈希输出的状态取出，这K个结结果中有一个未描黑即可判断不是非法url。 ==> 检索\n\n巧妙点是 确定 K、M的大小：M为bit的个数\n1. M的大小考虑相关：样本量N、失误率P。 M越大失误率越低，通过失误率P找到满足条件的M即可，无需无止境的扩大M，因为失误率无法=0，只能趋近。\nM =  -(N * lnP)\n    -----------\n       (ln2)^2\n2. K个函数的个数相关：样本量、M空间。初始K=1，P很大，随着K增大P会先变小，然后增大。增大是因为K增加使M的利用率增加，使失误率增加。\n所以是个凹函数，取拐点即可获得最佳K值。\nK = ln2 * M/N = 0.7 * M/N\n\n3. 真实P的计算方式：\nP = (1-e^(-N*K/M))^K\n\n4.通过2个哈希函数造K个：任意整数i * f1 + f2即可\n\n'

### 一致性哈希
分布式存储结构最常见的结构
1. 哈希域变成环的设计
2. 虚拟节点技术

In [ ]:
'''
1. 想象哈希函数的返回值(哈希值)组成一个环，将机器的某种特征通过这个哈希函数上环。比如3台机器的ip地址，哈希后，落到了环上三个点
2. 新的数据到来，通过哈希函数上环，存储到顺时针遇到的第一台机器。（每台机器存储一个有序数组即可）
3. 新添加一台机器，可以通过上环的位置，向老机器要到需要属于自己的分摊数据。
4. 这样对比所有数据的哈希值通过mod机器数，决定存储那台机器的方法：新加入机器需要重新哈希分配。而上面的方法不需要。

问题1:
负载可能不均：因为哈希均匀性是指一定数量的数据通过哈希hash后，命中每个桶的数量差不多；而不是会将几个点均匀地分布在环上。
            所以有可能存在某个机器负载很低，有的却很高。
解决方法：虚拟节点技术
1. 使用虚拟节点去上环抢数据！实际是通过虚拟节点数代替机器数上环，让环的分配更均衡。
2. 通过机器的性能调整，每台机器的虚拟节点个数。所以不光能解决负载不均，也可以解决性能管理。
'''